In [1]:
from fbx import *
import fbx

sample_fbx_path = "E:/projects/the_witcher/content_pipeline/fbx_samples/nathan/rp_nathan_animated_003_walking.fbx"

In [10]:
# ---------------- FBX WRAPPERS AND HELPERS ---------------- #
import os
from typing import List

# this fbx scene wrapper class will automatically destroy objects in memory
class FbxSceneWrapper:
    
    def __init__(self, filepath):
        self.mem_manager = fbx.FbxManager.Create()
        self.scene = fbx.FbxScene.Create(self.mem_manager, "")
        self.importer = fbx.FbxImporter.Create(self.mem_manager, "")
        self.importer.Initialize(os.path.abspath(filepath), -1, self.mem_manager.GetIOSettings())
        self.importer.Import(self.scene)
        
    def __enter__(self) -> fbx.FbxScene:
        return self.scene
    
    def __exit__(self, exc_type, exc_value, traceback):
        self.mem_manager.Destroy()    

# this helper class assists doing recursions of different flavours
class TreeShapedRecursion:
    
    def __init__(self, get_branches, do_with_each_branch):
        # should be a procedure to get child branches of a branch
        self.get_branches = get_branches 
        # should a procedure taking 1) branch and 2) recursion depth as arguments, performed on each branch
        self.do_with_each_branch = do_with_each_branch 
        self.current_depth = 0
    
    def __recursive_do__(self, root, current_recursion_depth=0):
        self.do_with_each_branch(root, current_recursion_depth)
        for branch in self.get_branches(root):
            self.__recursive_do__(branch, current_recursion_depth + 1)
            
    def __call__(self, root):
        self.__recursive_do__(root)

# an class that represents an enum, where enum members are supplied by an fbx type
class FbxEnum:
    
    def __init__(self, cls):
        if cls is not None:
            self.data = [(memb, getattr(cls, memb)) for memb in dir(cls) if memb.startswith('e')]
    
    @classmethod
    def from_data(cls, data):
        enum = FbxEnum(None)
        enum.data = data
        return enum
        
    def str(self, en_str) -> int:
        return {str_int_pair[0]: str_int_pair[1] for str_int_pair in self.data.items()}.get(en_str, -1)
    
    def int(self, en_int) -> str:
        return {str_int_pair[1]: str_int_pair[0] for str_int_pair in self.data.items()}.get(en_int, '')


NODE_ATTRIBUTE_TYPE = FbxEnum.from_data(
    {
        'unknown': int(fbx.FbxNodeAttribute.eUnknown),
        'null': int(fbx.FbxNodeAttribute.eNull),
        'marker': int(fbx.FbxNodeAttribute.eMarker),
        'skeleton': int(fbx.FbxNodeAttribute.eSkeleton),
        'mesh': int(fbx.FbxNodeAttribute.eMesh),
        'nurbs': int(fbx.FbxNodeAttribute.eNurbs),
        'patch': int(fbx.FbxNodeAttribute.ePatch),
        'camera': int(fbx.FbxNodeAttribute.eCamera),
        'camera_stereo': int(fbx.FbxNodeAttribute.eCameraStereo),
        'camera_switcher': int(fbx.FbxNodeAttribute.eCameraSwitcher),
        'light': int(fbx.FbxNodeAttribute.eLight),
        'optical_reference': int(fbx.FbxNodeAttribute.eOpticalReference),
        'optical_marker': int(fbx.FbxNodeAttribute.eOpticalMarker),
        'nurbs_curve': int(fbx.FbxNodeAttribute.eNurbsCurve),
        'trim_nurbs_surface': int(fbx.FbxNodeAttribute.eTrimNurbsSurface),
        'boundary': int(fbx.FbxNodeAttribute.eBoundary),
        'nurbs_surface': int(fbx.FbxNodeAttribute.eNurbsSurface),
        'shape': int(fbx.FbxNodeAttribute.eShape),
        'lod_group': int(fbx.FbxNodeAttribute.eLODGroup),
        'sub_div': int(fbx.FbxNodeAttribute.eSubDiv),
        'cached_effect': int(fbx.FbxNodeAttribute.eCachedEffect),
        'line': int(fbx.FbxNodeAttribute.eLine)
    }
)

# open an fbx scene as a closeable resource
def fbx_open(filepath):
    return FbxSceneWrapper(filepath)

# briefly describes an fbx node
def describe_fbx_node(node: fbx.FbxNode):
    nd_attr: fbx.FbxNodeAttribute = None
    nd_attr_type = -1
    if node.GetNodeAttributeCount() > 0:
        nd_attr = node.GetNodeAttributeByIndex(0)
        nd_attr_type: int = NODE_ATTRIBUTE_TYPE.int(int(nd_attr.GetAttributeType()))
    return 'name: {}, type: {}'.format(node.GetName(), nd_attr_type) 

# gets children of an fbx node as an iterable
def get_node_children(node: fbx.FbxNode):
    children = []
    for i in range(0, node.GetChildCount()):
        children.append(node.GetChild(i))
    return children

# get all nodes in a flat structure
def get_nodes(scene: fbx.FbxScene) -> List[fbx.FbxNode]:
    out_nodes = []
    TreeShapedRecursion(get_node_children, lambda nd, dpth: out_nodes.append(nd))(scene.GetRootNode())
    return out_nodes

# turn fbx array into a list
def listify(fbx_array):
    if isinstance(fbx_array, fbx.FbxVector4):
        array_4: fbx.FbxVector4 = fbx_array
        out = []
        for i in range(0, array_4.Length

In [14]:
# print all nodes of the file as a tree
with fbx_open(sample_fbx_path) as scene:
    TreeShapedRecursion(
        get_node_children,
        lambda node, depth: print('-- '*depth + describe_fbx_node(node))
    )(scene.GetRootNode())

name: RootNode, type: -1
-- name: rp_nathan_animated_003_walking, type: null
-- -- name: rp_nathan_animated_003_walking_CTRL, type: line
-- -- -- name: rp_nathan_animated_003_walking_root, type: skeleton
-- -- -- -- name: rp_nathan_animated_003_walking_hip, type: skeleton
-- -- -- -- -- name: rp_nathan_animated_003_walking_spine_01, type: skeleton
-- -- -- -- -- -- name: rp_nathan_animated_003_walking_spine_02, type: skeleton
-- -- -- -- -- -- -- name: rp_nathan_animated_003_walking_spine_03, type: skeleton
-- -- -- -- -- -- -- -- name: rp_nathan_animated_003_walking_neck, type: skeleton
-- -- -- -- -- -- -- -- -- name: rp_nathan_animated_003_walking_head, type: skeleton
-- -- -- -- -- -- -- -- -- -- name: rp_nathan_animated_003_walking_head_end, type: skeleton
-- -- -- -- -- -- -- -- -- -- name: rp_nathan_animated_003_walking_jaw, type: skeleton
-- -- -- -- -- -- -- -- -- -- -- name: rp_nathan_animated_003_walking_jaw_end, type: skeleton
-- -- -- -- -- -- -- -- -- -- name: rp_nathan_a

In [23]:
# display the vertices retrieved from the mesh
with fbx_open(sample_fbx_path) as scene:
    nodes = get_nodes(scene)
    mesh_node: fbx.FbxNode = [nd for nd in nodes if nd.GetName() == 'rp_nathan_animated_003_walking_geo'][0]
    mesh: fbx.FbxMesh = mesh_node.GetMesh()
    print([[v[0], v[1], v[2], v[3]] for v in mesh.GetControlPoints()])

[[0.17621295154094696, 165.55722045898438, 11.123315811157227, 0.0], [0.08345109224319458, 165.3172149658203, 11.135693550109863, 0.0], [0.18181240558624268, 165.31507873535156, 11.127449989318848, 0.0], [0.12461373209953308, 164.86041259765625, 11.111424446105957, 0.0], [0.19021452963352203, 164.95465087890625, 11.121068954467773, 0.0], [-2.121251106262207, 165.38279724121094, 8.916138648986816, 0.0], [-2.170361042022705, 165.41250610351562, 8.677984237670898, 0.0], [-2.118765354156494, 165.09725952148438, 8.700471878051758, 0.0], [-2.102860689163208, 165.06268310546875, 8.932344436645508, 0.0], [2.4027767181396484, 166.40264892578125, 9.299911499023438, 0.0], [2.368378162384033, 166.8820037841797, 9.283645629882812, 0.0], [2.2075209617614746, 166.84771728515625, 10.053062438964844, 0.0], [2.2161288261413574, 166.34762573242188, 10.066171646118164, 0.0], [-2.050595998764038, 165.04393005371094, 9.194235801696777, 0.0], [-2.079525947570801, 165.36563110351562, 9.159781455993652, 0.0], 